## Домашнее задание №3 ##

<ol>
<li>Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log (как вариант - использовать np.clip или np.where).
<li>Подберите аргументы функции eval_LR_model для логистической регрессии таким образом, чтобы log loss был минимальным (оптимально здесь использовать критерий остановки обучения).
<li>Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, на выходе - массив y_pred_proba).
<li>Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются значения признаков Х и веса, которые уже посчитаны функцией eval_LR_model, на выходе - массив y_pred).
<li>Посчитайте accuracy, матрицу ошибок, precision и recall, а также F1-score.
<li>Могла ли модель переобучиться? Почему?
<li>(*) Создайте функции eval_LR_model_l1 и eval_LR_model_l2 с применением L1 и L2 регуляризации соответственно.
</ol>

In [1]:
import numpy as np

In [2]:
# К признаку стажа добавим новые признаки
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64).T # квалификация репетитора
               
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64) # поступил или нет ученик на специальность Математика

In [3]:
def standard_scale(x):
    res = (x - x.mean(axis=0)) / x.std(axis=0)
    return res

In [4]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res


In [5]:
X_st = X.copy()
X_st[:, 1:4] = standard_scale(X_st[:, 1:4])
X_st

array([[ 1.        , -0.57142857, -0.97958969, -0.89625816],
       [ 1.        , -0.57142857, -0.56713087, -0.89625816],
       [ 1.        , -0.21428571, -0.46401617,  0.38411064],
       [ 1.        , -0.57142857, -0.77336028, -0.89625816],
       [ 1.        ,  0.14285714,  0.97958969,  0.38411064],
       [ 1.        , -0.92857143, -0.36090146, -0.89625816],
       [ 1.        ,  0.85714286,  1.08270439,  1.66447944],
       [ 1.        ,  2.64285714,  2.11385144,  1.66447944],
       [ 1.        , -0.57142857, -1.08270439, -0.89625816],
       [ 1.        , -0.21428571,  0.05155735,  0.38411064]])

## Задание 1

**Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.**

### Решение

In [6]:


def calc_logloss(y, y_pred):
    delta = 0.00001
    y_pred = np.clip(y_pred, delta, 1 - delta)
    #print(y_pred)
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    return err

In [7]:
y_pred = np.array([0.001, 0, 0.99, 0, 0.9, 0, 0.8, 0, 0.9999, 1])
calc_logloss(y, y_pred)

0.033970490840945576

## Задание 2

Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

### Решение

In [8]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],  # стаж репетитора
              [500, 700, 750, 600, 1450,        # средняя стоимость занятия
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 1, 3, 3, 1, 2]], dtype = np.float64).T # квалификация репетитора
               
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64) # поступил или нет ученик на специальность Математика

In [9]:
np.random.seed(42)

def eval_model(X, y, iterations, alpha=1e-4):
    np.random.seed(42)
    w = np.random.randn(X.shape[1])
    n = X.shape[0]
    for i in range(1, iterations + 1):
        z = np.dot(X, w) # log(p/(1-p))
        y_pred = sigmoid(z) # p [0, 1]
        err = calc_logloss(y, y_pred)
        w -= alpha * (1/n * np.dot(X.T, (y_pred - y)))
        #if i % (iterations / 100) == 0:
            #print(i, w, err)
    print(f'iterations = {iterations} , alpha = {alpha}, W = {w}, err = {err}')
    err = calc_logloss(y, y_pred)
    return iterations,alpha, w,err


In [10]:
X_st = X.copy()
X_st[:, 1:4] = standard_scale(X_st[:, 1:4])
alphas = np.logspace(-5, 1, 7)
iterations = np.arange(1000, 15000, 1500)

In [11]:
alphas, iterations, X_st

(array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01]),
 array([ 1000,  2500,  4000,  5500,  7000,  8500, 10000, 11500, 13000,
        14500]),
 array([[ 1.        , -0.57142857, -0.97958969, -0.89625816],
        [ 1.        , -0.57142857, -0.56713087, -0.89625816],
        [ 1.        , -0.21428571, -0.46401617,  0.38411064],
        [ 1.        , -0.57142857, -0.77336028, -0.89625816],
        [ 1.        ,  0.14285714,  0.97958969,  0.38411064],
        [ 1.        , -0.92857143, -0.36090146, -0.89625816],
        [ 1.        ,  0.85714286,  1.08270439,  1.66447944],
        [ 1.        ,  2.64285714,  2.11385144,  1.66447944],
        [ 1.        , -0.57142857, -1.08270439, -0.89625816],
        [ 1.        , -0.21428571,  0.05155735,  0.38411064]]))

In [12]:
eval_model(X_st, y, 10000, 0.0001)

iterations = 10000 , alpha = 0.0001, W = [ 0.44881007 -0.37237626  0.43601001  1.42113302], err = 0.6511273656414657


(10000,
 0.0001,
 array([ 0.44881007, -0.37237626,  0.43601001,  1.42113302]),
 0.6511273656414657)

In [13]:
results = []
for iteration in iterations:
    for alpha in alphas:
        results.append(eval_model(X_st, y, iteration, alpha))

iterations = 1000 , alpha = 1e-05, W = [ 0.49635334 -0.14077361  0.64537114  1.52180641], err = 0.7625205365285943
iterations = 1000 , alpha = 0.0001, W = [ 0.49298574 -0.16321527  0.62468762  1.51097412], err = 0.7507528681143767
iterations = 1000 , alpha = 0.001, W = [ 0.44881796 -0.37239011  0.43599362  1.42111651], err = 0.6512037362503481
iterations = 1000 , alpha = 0.01, W = [ 0.09249613 -1.16047276 -0.12476714  1.70921775], err = 0.46129017053513033
iterations = 1000 , alpha = 0.1, W = [ 0.37731566 -2.73985947 -1.18135067  4.58774123], err = 0.31008783816773305
iterations = 1000 , alpha = 1.0, W = [ 2.48597049 -6.07038967 -5.35080865 13.29834441], err = 0.17912798140855035
iterations = 1000 , alpha = 10.0, W = [  7.67909566 -14.87816361 -16.60836144  37.16064425], err = 0.07609858483095079
iterations = 2500 , alpha = 1e-05, W = [ 0.49580707 -0.14453168  0.64190222  1.51997866], err = 0.7605341625423251
iterations = 2500 , alpha = 0.0001, W = [ 0.4869033  -0.20003792  0.59091831 

In [14]:
results

[(1000,
  1e-05,
  array([ 0.49635334, -0.14077361,  0.64537114,  1.52180641]),
  0.7625205365285943),
 (1000,
  0.0001,
  array([ 0.49298574, -0.16321527,  0.62468762,  1.51097412]),
  0.7507528681143767),
 (1000,
  0.001,
  array([ 0.44881796, -0.37239011,  0.43599362,  1.42111651]),
  0.6512037362503481),
 (1000,
  0.01,
  array([ 0.09249613, -1.16047276, -0.12476714,  1.70921775]),
  0.46129017053513033),
 (1000,
  0.1,
  array([ 0.37731566, -2.73985947, -1.18135067,  4.58774123]),
  0.31008783816773305),
 (1000,
  1.0,
  array([ 2.48597049, -6.07038967, -5.35080865, 13.29834441]),
  0.17912798140855035),
 (1000,
  10.0,
  array([  7.67909566, -14.87816361, -16.60836144,  37.16064425]),
  0.07609858483095079),
 (2500,
  1e-05,
  array([ 0.49580707, -0.14453168,  0.64190222,  1.51997866]),
  0.7605341625423251),
 (2500,
  0.0001,
  array([ 0.4869033 , -0.20003792,  0.59091831,  1.49364441]),
  0.7318604267936508),
 (2500,
  0.001,
  array([ 0.35255638, -0.64701881,  0.20326298,  1.3

In [15]:
min = [None,None,None, np.inf]
for result in results:
    if result[3] < min[3]:
        min = result

In [16]:
print(f'Interation = {min[0]}, aplha = {min[1]}, W = {min[2]}, ERROR = {min[3]}')

Interation = 14500, aplha = 10.0, W = [ 21.03884034 -35.62212972 -52.6484788  106.72947318], ERROR = 0.013419759526532304


## Задание 3

Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).


### Решение

In [19]:
def calc_pred_proba(W, X):
    y_pred_proba = sigmoid(np.dot(W, X.T))
    return y_pred_proba

In [20]:
y_pred_proba = calc_pred_proba(min[2], X_st)
y_pred_proba

array([6.36308625e-02, 2.51984048e-11, 1.00000000e+00, 1.30856806e-06,
       9.95376568e-01, 1.62601959e-10, 1.00000000e+00, 1.17254149e-03,
       9.39341604e-01, 1.00000000e+00])

## Задание 4

Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

### Решение

In [21]:
def calc_pred(min[2], X, treshold=0.5):
    y_pred = sigmoid(np.dot(W, X.T)) > treshold
    return y_pred.astype(int)

SyntaxError: invalid syntax (<ipython-input-21-2e7da51787ce>, line 1)

In [22]:
y_pred = calc_pred(min[2], X_st)
y_pred

NameError: name 'calc_pred' is not defined

## Задание 5

*Реализуйте функции для подсчета Accuracy, матрицы ошибок, точности и полноты, а также F1 score.

### Решение

In [23]:
def calc_accuracy(y, y_pred):
    total_successes = sum(y_pred==y)
    return total_successes/len(y)

calc_accuracy(y, y_pred)

0.5

In [24]:
def calc_matrix(y, y_pred):
    pairs = zip(y, y_pred)
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for origin, pred in pairs:
        if pred and origin:
            true_positive+= 1
        elif pred and not origin:
            false_negative += 1
        elif origin and not pred:
            false_positive += 1
        else:
            true_negative +=1

    return true_positive, true_negative, false_positive, false_negative

In [25]:
def calc_recall(matrix):
    true_positive = matrix[0]
    false_negative = matrix[3]

    return true_positive / (true_positive + false_negative)

def calc_precision(matrix):
    true_positive = matrix[0]
    false_positive = matrix[2]

    return true_positive / (true_positive + false_positive)

def calc_f_score(precision, recall):
    return 2 * recall * precision  / (recall + precision)

In [26]:
matrix = calc_matrix(y, y_pred)
matrix

(5, 4, 0, 1)

In [27]:
recall = calc_recall(matrix)
recall

0.8333333333333334

In [28]:
precision = calc_precision(matrix)
precision

1.0

In [29]:
calc_f_score(precision, recall)

0.9090909090909091

## Задание 6

Могла ли модель переобучиться? Почему?

### Решение

Вероятно не могла - очень мало данных